# Visualize Anatomical Modules

### This tool allows the visualization of Scaffold 3D models 
### Three options are supported for importing the data

### Option 1 : Import VTK files 
### Option 2 : Visualise Anatomical Module from the SPARC Portal
### Option 3 : Use Demonstration Data

In [ ]:
import sys
sys.stdout = open('/dev/null', 'w')
!pip install -r requirements.txt
!pip install open3d
!pip install stl
!pip install vtk
!pip install open3d
!pip install vtk
!pip install numpy-stl
!pip install pywavefront

In [3]:
from stl import mesh
import ipywidgets as widgets
from IPython.display import display, clear_output, HTML
import open3d as o3d
import numpy as np
import plotly.graph_objects as go
import pandas as pd
import vtk
from stl import mesh
from pywavefront import Wavefront
import os

# Read internal data
df = pd.read_csv('Internal data/IDs_data.csv')

def generate_stl_file():
    input_vtk = "input data/scaffold.vtk"
    # Read the VTK file
    reader = vtk.vtkUnstructuredGridReader()
    reader.SetFileName(input_vtk)
    reader.Update()

    # Convert to polydata
    geometryFilter = vtk.vtkGeometryFilter()
    geometryFilter.SetInputConnection(reader.GetOutputPort())
    geometryFilter.Update()

    # Write as STL
    stlWriter = vtk.vtkSTLWriter()
    stlWriter.SetFileName('output data/scaffold.stl') 
    stlWriter.SetInputConnection(geometryFilter.GetOutputPort())
    stlWriter.Write()

def vtk_to_stl(vtk_filename, stl_filename):
    # Read VTK file
    reader = vtk.vtkPolyDataReader()
    reader.SetFileName(vtk_filename)
    reader.Update()
    
    # Convert VTK to STL
    poly_data = reader.GetOutput()
    stl_writer = vtk.vtkSTLWriter()
    stl_writer.SetFileName(stl_filename)
    stl_writer.SetInputData(poly_data)
    stl_writer.Write()

def stl_to_obj(stl_filename, obj_filename):
    # Read STL file
    stl_mesh = mesh.Mesh.from_file(stl_filename)
    
    # Create OBJ file
    with open(obj_filename, 'w') as obj_file:
        # Write vertices
        for vertex in stl_mesh.vectors.reshape(-1, 3):
            obj_file.write(f"v {vertex[0]} {vertex[1]} {vertex[2]}\n")
        
        # Write faces
        num_faces = len(stl_mesh.vectors)
        for i in range(num_faces):
            obj_file.write(f"f {3*i+1} {3*i+2} {3*i+3}\n")

def vtk_to_obj(vtk_filename, obj_filename):
    stl_filename = "output data/scaffold.stl"
    
    # Convert VTK to STL
    vtk_to_stl(vtk_filename, stl_filename)
    
    # Convert STL to OBJ
    stl_to_obj(stl_filename, obj_filename)


# Define the toggle buttons (mutually exclusive selection)
toggle_buttons = widgets.ToggleButtons(
    options=[ 'Demo', 'Import VTK file', 'SPARC Portal Dataset'],
    description='Select Option:',
    disabled=False,
    button_style='',  # Can be 'success', 'info', 'warning', 'danger' for different styles
    tooltips=['Use demo data', 'Select dataset from SPARC Portal', 'Import a VTK file'],
)

# Create the dropdown widgets for Species and Organs
species_options = ['human', 'mouse', 'rat', 'pig', 'sheep', 'cat']
species_dropdown = widgets.Dropdown(
    options=species_options,
    value=species_options[0],  # Set default value
    description='Species:',
    disabled=False,
)

organ_options = ['whole_body', 'stomach', 'colon', 'brainstem', 'heart', 'lung', 'esophagus']
organ_dropdown = widgets.Dropdown(
    options=organ_options,
    value=organ_options[0],  # Set default value
    description='Organ:',
    disabled=False,
)

# Button to submit the selection
button = widgets.Button(description='Submit')
output = widgets.Output()

# Variable to store the inserted ID
ID = None

# Function to update the displayed widgets based on the selected option
def update_widgets(*args):
    selected_option = toggle_buttons.value
    
    # Update dropdowns visibility
    if selected_option == 'SPARC Portal Dataset':
        dropdown_container.children = [species_dropdown, organ_dropdown]  # Show both dropdowns if SPARC Portal Dataset is selected
    else:
        dropdown_container.children = []  # Hide the dropdowns for other selections

    # Update file upload visibility
    if selected_option == 'Import VTK file':
        file_upload_container.children = [file_upload]  # Show file upload widget if Import VTK file is selected
    else:
        file_upload_container.children = []  # Hide file upload widget for other selections

# Attach the update function to the toggle buttons
toggle_buttons.observe(update_widgets, names='value')

# Function to handle the button click event
def on_button_click(b):
    global ID
    with output:
        output.clear_output()  # Clear previous output
        selected_option = toggle_buttons.value
        
        if selected_option == 'Demo':
            print("Loading demonstration data...")
            mesh = o3d.io.read_triangle_mesh("Internal data/stomac.obj")
            
        elif selected_option == 'Import VTK file':
            if file_upload.value:
                # Process the file upload
                handle_file_upload()
                generate_stl_file()
                vtk_filename = "input data/scaffold.vtk"
                obj_filename = "scaffold.obj"
                vtk_to_obj(vtk_filename, obj_filename)
                mesh = o3d.io.read_triangle_mesh("scaffold.obj")
            else:
                print("No file uploaded. Please upload a VTK file using the file upload widget.")
        
        elif selected_option == 'SPARC Portal Dataset':
            print("The table below includes datasets that correspond to your selected species and organ:")
            # Filter DataFrame based on dropdown values
            filtered_df = df[(df['bodypart'] == organ_dropdown.value) & (df['species'] == species_dropdown.value)]
            columns_to_keep = [df.columns[0], df.columns[1], df.columns[6], df.columns[7]]
            filtered_df = filtered_df[columns_to_keep]
            display(HTML(filtered_df.head().to_html(index=False)))  # Display the filtered dataframe
            print("To visualize the scaffold data insert the ID of the dataset in the box below.")

            # Create an input box and label to display the input and result
            input_box = widgets.Text(description='Insert ID:', placeholder='Enter dataset ID')
            output_label = widgets.Label(value='')
            result_label = widgets.Label(value='')
            display(input_box)
            display(output_label)
            display(result_label)
            
            def on_input_change(change):
                try:
                    dataset_id = int(change.new)  # Convert input to integer
                    file_extension = '.obj'
                    search_dir = 'Internal data/'

                    # Construct the file name
                    file_name = f'{dataset_id}{file_extension}'
                    # Search for the file in the search directory
                    file_found = False
                    for root, dirs, files in os.walk(search_dir):
                        if file_name in files:
                            file_path = os.path.join(root, file_name)
                            print(f'File found: {file_path}')
                            file_found = True

                            # Read the file using o3d.io.read_triangle_mesh()
                            mesh = o3d.io.read_triangle_mesh(file_path)
                            print("mesh created")
                            result_label.value = 'Mesh created successfully!'
                            break
                    if not file_found:
                        print(f'File not found: {file_name}')
                        result_label.value = f'File not found: {file_name}'
                except ValueError:
                    output_label.value = 'Please enter a valid number.'
                    result_label.value = ''

            input_box.observe(on_input_change, names='value')


        else:
                print("No Data source is defined. Please choose an option.")
       
    
    #####################
        if 'mesh' in locals() and not mesh.is_empty(): 
            if not mesh.has_vertex_normals(): 
                mesh.compute_vertex_normals()
            if not mesh.has_triangle_normals(): 
                mesh.compute_triangle_normals()
            triangles = np.asarray(mesh.triangles)
            vertices = np.asarray(mesh.vertices)
            colors = None
            if mesh.has_triangle_normals():
                colors = (0.5, 0.5, 0.5) + np.asarray(mesh.triangle_normals) * 0.5
                colors = tuple(map(tuple, colors))
            else:
                colors = (1.0, 0.0, 0.0)
                
            fig = go.Figure(
                data=[
                    go.Mesh3d(
                        x=vertices[:,0],
                        y=vertices[:,1],
                        z=vertices[:,2],
                        i=triangles[:,0],
                        j=triangles[:,1],
                        k=triangles[:,2],
                        facecolor=colors)
                ],
                layout=dict(
                    scene=dict(
                        xaxis=dict(visible=False),
                        yaxis=dict(visible=False),
                        zaxis=dict(visible=False)
                    )
                )
            )
            fig.show(width=10000, height=10000)     
        else:
            print("Mesh is empty or not available")
            

# Function to handle file upload and save it to the folder
def handle_file_upload():
    with output:
        output.clear_output()  # Clear previous output
        
        if file_upload.value:
            # Get the uploaded file content
            uploaded_file = file_upload.value[0]  # Access the tuple directly
            uploaded_filename = uploaded_file['name']
            content = uploaded_file['content']  # Access the actual file content
            
            # Save the file to the specified directory
            file_path = os.path.join(save_dir, uploaded_filename)
            with open(file_path, "wb") as f:
                f.write(content)
                # Rename the file to scaffold.vtk
                new_file_path = os.path.join(save_dir, 'scaffold.vtk')
                os.rename(file_path, new_file_path)
            
            print("File successfully imported")
        else:
            print("No file uploaded")

# Attach the button click event to the function
button.on_click(on_button_click)

# Container to hold the dropdown widgets (empty initially)
dropdown_container = widgets.VBox([])

# Create a FileUpload widget
file_upload = widgets.FileUpload(
    accept='.vtk,.stl,.obj',  # Accept VTK, STL, and OBJ files
    multiple=False  # Only allow single file uploads
)

# Container to hold the file upload widget (empty initially)
file_upload_container = widgets.VBox([])

# Create an output widget to display messages
output = widgets.Output()

# Directory to save the uploaded file
save_dir = "input data"

# Create the directory if it doesn't exist
if not os.path.exists(save_dir):
    os.makedirs(save_dir)

# Attach the function to the FileUpload widget
file_upload.observe(lambda change: None, names='value')  # No action needed on file upload alone
#
# Create the sidebar widget with the dropdown menus and button
sidebar_widget = widgets.VBox([ 
    toggle_buttons,
    dropdown_container,
    file_upload_container,
    button
])

# Create the main panel widget
main_panel_widget = widgets.VBox([
    #widgets.Label('The results include a table, a simplified body map illustrating the neurons between the regions, and a simplified plot illustration the intermidiate regions'),
    output
])

# Create a HBox to arrange the sidebar and main panel side by side
layout_widget = widgets.HBox([
    widgets.VBox([
        widgets.HTML("<h3>Please Select target data</h3>"),
        sidebar_widget,
    ], layout=widgets.Layout(width='25%', background_color='#f0f0f0')),
    widgets.VBox([
        widgets.HTML("<h3></h3>"),
        main_panel_widget
    ], layout=widgets.Layout(width='75%', background_color='#ffffff'))
])
# Display the layout widget
display(layout_widget)

# Initial call to set up the widgets based on default selection
update_widgets()
